In [66]:
import torch
from torch import nn
import os
import re
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import numpy as np
import collections
import nltk
from d2l import torch as d2l
import math
import torch.nn.functional as F

In [67]:
fileDir='/home/li/ML_object/LSTM_imdb/aclImdb'

In [68]:
strip_special_chars = re.compile("[^A-Za-z ]+")
def tokenize(content):
    content = re.sub('<.*?>', ' ', content)
#     fileters =['\/', '\(', '\)', ':', '\.', '\t', '\n', '\x97', '\x96', '#', '$', '%', '&']
#     content = re.sub('|'.join(fileters), ' ', content)
    content = re.sub(strip_special_chars, "", content)
    tokens = [i.strip().lower() for i in content.split()]
    return tokens

In [69]:
class IMDBData(Dataset):
    def __init__(self,train=True):
        if train:
            data_path=os.path.join(fileDir,"train")
        else:
            data_path=os.path.join(fileDir,"test")
        allPath=[os.path.join(data_path,"neg"),os.path.join(data_path,"pos")]
        self.data_path=[]
        for path in allPath:
            pathName=os.listdir(path)
            path_list=[os.path.join(path,name) for name in pathName]
            self.data_path.extend(path_list)         
        
            
    def __getitem__(self,index):
        path=self.data_path[index]
        label_str=path.split('/')[-2]
        label=0 if label_str=='neg' else 1
        content=open(path).read()
        token=tokenize(content)
        return token,label
        
    def __len__(self):
        return len(self.data_path)

In [70]:
trainset=IMDBData()
testset=IMDBData(train=False)

In [71]:
for path in trainset.data_path:
    content=open(path).read()
#     print(content)
    tokens=tokenize(content)
#     print(tokens)
    break

In [72]:
class word2idx:
    def __init__(self):
        self.cnt=0
        self.word_to_idx={}
        self.word_cnt={}
    
    def add_word(self,word):
        if not word in self.word_to_idx:
            self.word_to_idx[word]=self.cnt
            self.word_cnt[word]=1
            self.cnt+=1
        else:
            self.word_cnt[word]+=1
    
    def add_text(self,test):
        for word in test:
            self.add_word(word)
            

In [73]:
def limitDict(limit,classObj) :
    dict1 = sorted(classObj.word_cnt.items(),key = lambda t : t[1], reverse = True)
#     print(dict1)
    count = 0
    for x,y in dict1 :
        if count >= limit-1 :
            classObj.word_to_idx[x] = limit
        else :
            classObj.word_to_idx[x] = count

        count+=1

In [74]:
vocabLimit = 50000
max_sequence_len = 300
obj1 = word2idx()
device=d2l.try_gpu()
print(device)

cuda:0


In [75]:
print("reading all train test")
data_path=np.array(trainset.data_path)
np.random.shuffle(data_path)
for path in data_path:
    content=open(path).read()
    tokens=tokenize(content)
    obj1.add_text(tokens)
print("read all texts")

reading all train test
read all texts


In [76]:
limitDict(vocabLimit,obj1)
# print(obj1.word_to_idx)

In [77]:
class RNNmodel(nn.Module):
    def __init__(self,embedding_dim,hidden_dim):
        super(RNNmodel,self).__init__()
        self.hidden_dim=hidden_dim
        self.embeddings=nn.Embedding(vocabLimit+1,embedding_dim)
        self.lstm=nn.LSTM(embedding_dim,self.hidden_dim)
        self.linear=nn.Linear(hidden_dim,2)
    
    def forward(self,inputs,state):
        x = self.embeddings(inputs).view(len(inputs),1,-1)
        y,state = self.lstm(x,state)
        output = y[-1]
        output = self.linear(output)
#         sfm=nn.Softmax(dim=1)
#         output = sfm(output)
        return output,state
    
    def begin_state(self,device,batch_size=1):
        return(torch.zeros((1,batch_size,self.hidden_dim),device=device),
              torch.zeros((1,batch_size,self.hidden_dim),device=device),)
    

In [78]:
net=RNNmodel(50,100)
seq_len=300

In [79]:
num_epochs=3
lr=0.008

In [80]:
loss = nn.CrossEntropyLoss()
updater=torch.optim.SGD(net.parameters(), lr)

In [81]:
torch.save(net.state_dict(), 'RNNmodel' + str(0)+'.pth')
print('starting training')

starting training


In [ ]:
# net.load_state_dict(torch.load('RNNmodel.pth'))

In [82]:
net.to(device)

RNNmodel(
  (embeddings): Embedding(50001, 50)
  (lstm): LSTM(50, 100)
  (linear): Linear(in_features=100, out_features=2, bias=True)
)

In [100]:
for i in range(num_epochs):
    avg_loss=0.0
    cnt=0
    data_path=np.array(trainset.data_path)
    np.random.shuffle(data_path)
    for path in data_path:
        cnt+=1
        content=open(path).read()
        input_data=tokenize(content)
        input_data=[obj1.word_to_idx[word] for word in input_data]
        if len(input_data) > seq_len:
            input_data=input_data[:seq_len]
        input_data = Variable(torch.cuda.LongTensor(input_data))
        if path.split('/')[-2] == 'neg':
            label=0
        else:
            label=1
        target = Variable(torch.cuda.LongTensor([label]))
#         print(target)
        state=net.begin_state(device)
#         print(state[0])
#         print(input_data)
        y_pred,_=net(input_data,state)
#         print(y_pred,target)
        net.zero_grad()
#         print(y_pred)
#         print("y ",target)
        l=loss(y_pred,target)
        avg_loss+=float(l)
        if cnt%5000==0:
            print('epoch :%d iterations :%d loss :%g'%(i,cnt,float(l)))
        l.backward()
        updater.step()
        
#     torch.save(net.state_dict(), 'RNNmodel' + str(i+1)+'.pth')
    print('the average loss after completion of %d epochs is %g'%((i+1),(avg_loss/len(trainset.data_path))))
torch.save(net.state_dict(), 'RNNmodel' +'.pth')


epoch :0 iterations :5000 loss :3.45706e-06
epoch :0 iterations :10000 loss :0.000910344
epoch :0 iterations :15000 loss :0.0131458
epoch :0 iterations :20000 loss :2.38418e-06
epoch :0 iterations :25000 loss :2.98023e-06
the average loss after completion of 1 epochs is 0.0480991
epoch :1 iterations :5000 loss :0.00182652
epoch :1 iterations :10000 loss :0.0010554
epoch :1 iterations :15000 loss :0.00322287
epoch :1 iterations :20000 loss :0.000127427
epoch :1 iterations :25000 loss :0
the average loss after completion of 2 epochs is 0.0478237
epoch :2 iterations :5000 loss :6.07966e-06
epoch :2 iterations :10000 loss :0.00986557
epoch :2 iterations :15000 loss :0.00128639
epoch :2 iterations :20000 loss :0.000297383
epoch :2 iterations :25000 loss :0.00407817
the average loss after completion of 3 epochs is 0.0307753


In [96]:
print("train over,start predict")

train over,start predict


In [103]:
obj2 = word2idx()
print("reading all test test")
for path in testset.data_path:
    content=open(path).read()
    tokens=tokenize(content)
    obj2.add_text(tokens)
print("read all texts")
limitDict(vocabLimit,obj2)

reading all test test
read all texts


In [ ]:
right_num=0
data_path=np.array(testset.data_path)
np.random.shuffle(data_path)
for path in data_path:
    cnt+=1
    content=open(path).read()
#     print(path)

#     data=tokenize(content)
#     print(input_data)
#     input_data=[]
#     for word in data:
#         if word not in obj1.word_to_idx:
#             input_data.append(vocabLimit)
#         else:
#             input_data.append(obj2.word_to_idx[word])
#     if len(input_data) > seq_len:
#         input_data=input_data[:seq_len]

    input_data=tokenize(content)
    input_data=[obj2.word_to_idx[word] for word in input_data]
    if len(input_data) > seq_len:
        input_data=input_data[:seq_len]
    input_data = Variable(torch.cuda.LongTensor(input_data))
    if path.split('/')[-2] == 'neg':
        label=0
    else:
        label=1
    target = Variable(torch.cuda.LongTensor([label]))
#    print(target)
    state=net.begin_state(device)
#         print(state[0])
#         print(input_data)
    y_pred,_=net(input_data,state)
#     print("gg: ",y_pred)
    pred1 = y_pred.data.max(1)[1].cpu().numpy()
    print(pred1,target)
    if int(pred1) == int(target):
        right_num+=1
print(right_num)
acc=right_num/len(testset.data_path)
#         print(y_pred)
#         print("y ",target)
    

[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor

[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor

[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor

[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor

[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor

...

[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor

[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor

[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor

[1] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor

[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[0] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[1] tensor([1], device='cuda:0')
[1] tensor([1], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor([0], device='cuda:0')
[0] tensor

In [99]:
print(acc)

0.5632
